In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
import numpy as np

In [3]:
root_url = "https://www.amazon.com/s?k=apple+iphone&sprefix=apple+%2Caps%2C307&ref=nb_sb_ss_ts-doa-p_3_6"

HTTPS_HEADERS = (
    {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36", 
         "Accept-Language": "en-US, en;q=0.5"
    }
)


In [4]:
web_page = requests.get(root_url, headers=HTTPS_HEADERS)

In [5]:
web_page

<Response [200]>

In [7]:
soup = BeautifulSoup(web_page.content, "html.parser")

In [13]:
links = soup.find_all("a", attrs={"class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})

In [16]:
link = links[0].get("href")

In [17]:
product_link = "https://amazon.com" + link

In [18]:
product_link

'https://amazon.com/TracFone-iPhone-SE-5G-Black/dp/B09VY7RXJ9/ref=sr_1_1?keywords=apple+iphone&qid=1686064775&sprefix=apple+%2Caps%2C307&sr=8-1'

In [19]:
new_web_page = requests.get(product_link, headers=HTTPS_HEADERS)

In [20]:
new_web_page

<Response [200]>

In [21]:
new_soup = BeautifulSoup(new_web_page.content, "html.parser")

In [27]:
first_name = new_soup.find("span", attrs={"id": "productTitle"}).text.strip()

In [25]:
new_soup.find("span", attrs={"class": "a-price aok-align-center"})

<span class="a-price aok-align-center" data-a-color="base" data-a-size="xl"><span class="a-offscreen">$399.00</span><span aria-hidden="true"><span class="a-price-symbol">$</span><span class="a-price-whole">399<span class="a-price-decimal">.</span></span><span class="a-price-fraction">00</span></span></span>

In [28]:
first_price = new_soup.find("span", attrs={"class": "a-price aok-align-center"}).find(
    "span", attrs={"class": "a-offscreen"}
).text.strip()

In [33]:
rating_first = new_soup.find("span", attrs={"class": "a-icon-alt"}).text.strip()

In [34]:
print("The First Item is :- {}".format(first_name))
print("\n")
print("The Price :- {}".format(first_price))
print("\n")
print("The Ratings are :- {}".format(rating_first))

The First Item is :- TracFone Apple iPhone SE 5G (3rd Generation), 64GB, Black - Prepaid Smartphone (Locked)


The Price :- $399.00


The Ratings are :- 3.8 out of 5 stars


In [41]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm as tqdm


# Function to extract Product Title
def getTitleFunction(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string


# Function to extract Product Price
def getPriceFunction(soup):

    try:
        price = soup.find("span", attrs={"class": "a-price aok-align-center"}).find(
                "span", attrs={"class": "a-offscreen"}
            ).text.strip()
        
    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={"class": "a-offscreen"}).string.strip()

        except:
            price = ""

    return price


# Function to extract Product Rating
def getRatingFunction(soup):

    try:
        rating = soup.find("i", attrs={"class" : "a-icon a-icon-star a-star-4 cm-cr-review-stars-spacing-big"}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating


# Function to extract Number of User Reviews
def getReviewCountFunction(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count


# Function to extract Availability Status
def getAvailabilityFunction(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available


if __name__ == '__main__':

    root_url = "https://www.amazon.com/s?k=apple+iphone&sprefix=apple+%2Caps%2C307&ref=nb_sb_ss_ts-doa-p_3_6"

    HTTPS_HEADERS = (
        {
            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36", 
             "Accept-Language": "en-US, en;q=0.5"
        }
    )

    # HTTP Request
    webpage = requests.get(root_url, headers=HTTPS_HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in tqdm(links) :
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[]}
    
    # Loop for extracting product details from each link 
    print("Performing the Web Scrapping, please wait a while.....")
    for link in tqdm(links_list) :
        try :
            new_webpage = requests.get("https://www.amazon.com" + link, headers=HTTPS_HEADERS)
            
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")
            
            # Function calls to display all necessary product information
            d['title'].append(getTitleFunction(new_soup))
            d['price'].append(getPriceFunction(new_soup))
            d['rating'].append(getRatingFunction(new_soup))
            d['reviews'].append(getReviewCountFunction(new_soup))
            
        except Exception as exp :
            print("HTTPS Exception occured.....")
            print("Ignoring and trying to continue....")
            continue
            
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [55]:
amazon_df

,title,price,rating,reviews,availability
5,"Apple iPhone 11, 64GB, Black - Unlocked (Renewed)",$299.00,4.3 out of 5 stars,"38,383 ratings",Not Available
6,"Apple iPhone 8, US Version, 64GB, Space Gray -...",$130.00,4.3 out of 5 stars,"51,440 ratings",Not Available
7,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)",$229.00,4.5 out of 5 stars,"58,842 ratings",Not Available
8,"Apple iPhone SE 2nd Generation, US Version, 64...",$141.00,4.3 out of 5 stars,"10,988 ratings",Not Available
9,"Apple iPhone 12, 64GB, Blue - Fully Unlocked (...",$379.75,4.2 out of 5 stars,"8,076 ratings",Not Available
10,"Apple iPhone 8 Plus, 64GB, Space Gray - Unlock...",$179.99,4.4 out of 5 stars,"25,880 ratings",Not Available
11,"Apple iPhone XS, US Version, 64GB, Space Gray ...",$230.00,4.3 out of 5 stars,"17,581 ratings",Not Available
12,"Apple iPhone 11 Pro, US Version, 256GB, Space ...",$649.99,4.2 out of 5 stars,"18,124 ratings",Not Available
13,"Apple iPhone X, US Version, 64GB, Silver - Unl...",$219.89,4.2 out of 5 stars,"23,185 ratings",Not Available
14,"Apple iPhone 12 Pro, 256GB, Graphite - Fully U...",$558.82,4.2 out of 5 stars,"3,669 ratings",Not Available
